# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 15 2022 10:10AM,246319,21,409206597,"NBTY Global, Inc.",Released
1,Sep 15 2022 10:09AM,246498,10,MSP213977,"Methapharm, Inc.",Released
2,Sep 15 2022 10:07AM,246067,21,626793,"NBTY Global, Inc.",Released
3,Sep 15 2022 10:05AM,246302,21,409216403,"NBTY Global, Inc.",Released
4,Sep 15 2022 10:03AM,246301,21,409206608,"NBTY Global, Inc.",Released
5,Sep 15 2022 9:58AM,246495,10,PRF-37047,Bio-PRF,Released
6,Sep 15 2022 9:58AM,246495,10,PRF-37051,Bio-PRF,Released
7,Sep 15 2022 9:58AM,246495,10,PRF-37057,Bio-PRF,Released
8,Sep 15 2022 9:58AM,246495,10,PRF-37061,Bio-PRF,Released
9,Sep 15 2022 9:58AM,246495,10,PRF-37065,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,246494,Released,1
29,246495,Released,10
30,246496,Released,1
31,246497,Released,5
32,246498,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
246494,NaN,1.0
246495,NaN,10.0
246496,NaN,1.0
246497,NaN,5.0
246498,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246066,0.0,1.0
246067,0.0,1.0
246278,0.0,1.0
246282,0.0,1.0
246291,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246066,0,1
246067,0,1
246278,0,1
246282,0,1
246291,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246066,0,1
1,246067,0,1
2,246278,0,1
3,246282,0,1
4,246291,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246066,,1
1,246067,,1
2,246278,,1
3,246282,,1
4,246291,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 15 2022 10:10AM,246319,21,"NBTY Global, Inc."
1,Sep 15 2022 10:09AM,246498,10,"Methapharm, Inc."
2,Sep 15 2022 10:07AM,246067,21,"NBTY Global, Inc."
3,Sep 15 2022 10:05AM,246302,21,"NBTY Global, Inc."
4,Sep 15 2022 10:03AM,246301,21,"NBTY Global, Inc."
5,Sep 15 2022 9:58AM,246495,10,Bio-PRF
15,Sep 15 2022 9:55AM,246282,21,"NBTY Global, Inc."
16,Sep 15 2022 9:54AM,246497,20,"ACI Healthcare USA, Inc."
21,Sep 15 2022 9:54AM,246278,21,"NBTY Global, Inc."
22,Sep 15 2022 9:51AM,246066,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 15 2022 10:10AM,246319,21,"NBTY Global, Inc.",,1
1,Sep 15 2022 10:09AM,246498,10,"Methapharm, Inc.",,1
2,Sep 15 2022 10:07AM,246067,21,"NBTY Global, Inc.",,1
3,Sep 15 2022 10:05AM,246302,21,"NBTY Global, Inc.",,1
4,Sep 15 2022 10:03AM,246301,21,"NBTY Global, Inc.",,1
5,Sep 15 2022 9:58AM,246495,10,Bio-PRF,,10
6,Sep 15 2022 9:55AM,246282,21,"NBTY Global, Inc.",,1
7,Sep 15 2022 9:54AM,246497,20,"ACI Healthcare USA, Inc.",,5
8,Sep 15 2022 9:54AM,246278,21,"NBTY Global, Inc.",,1
9,Sep 15 2022 9:51AM,246066,21,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 15 2022 10:10AM,246319,21,"NBTY Global, Inc.",1,
1,Sep 15 2022 10:09AM,246498,10,"Methapharm, Inc.",1,
2,Sep 15 2022 10:07AM,246067,21,"NBTY Global, Inc.",1,
3,Sep 15 2022 10:05AM,246302,21,"NBTY Global, Inc.",1,
4,Sep 15 2022 10:03AM,246301,21,"NBTY Global, Inc.",1,
5,Sep 15 2022 9:58AM,246495,10,Bio-PRF,10,
6,Sep 15 2022 9:55AM,246282,21,"NBTY Global, Inc.",1,
7,Sep 15 2022 9:54AM,246497,20,"ACI Healthcare USA, Inc.",5,
8,Sep 15 2022 9:54AM,246278,21,"NBTY Global, Inc.",1,
9,Sep 15 2022 9:51AM,246066,21,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 15 2022 10:10AM,246319,21,"NBTY Global, Inc.",1,
1,Sep 15 2022 10:09AM,246498,10,"Methapharm, Inc.",1,
2,Sep 15 2022 10:07AM,246067,21,"NBTY Global, Inc.",1,
3,Sep 15 2022 10:05AM,246302,21,"NBTY Global, Inc.",1,
4,Sep 15 2022 10:03AM,246301,21,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 15 2022 10:10AM,246319,21,"NBTY Global, Inc.",1.0,NaN
1,Sep 15 2022 10:09AM,246498,10,"Methapharm, Inc.",1.0,NaN
2,Sep 15 2022 10:07AM,246067,21,"NBTY Global, Inc.",1.0,NaN
3,Sep 15 2022 10:05AM,246302,21,"NBTY Global, Inc.",1.0,NaN
4,Sep 15 2022 10:03AM,246301,21,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 15 2022 10:10AM,246319,21,"NBTY Global, Inc.",1.0,0.0
1,Sep 15 2022 10:09AM,246498,10,"Methapharm, Inc.",1.0,0.0
2,Sep 15 2022 10:07AM,246067,21,"NBTY Global, Inc.",1.0,0.0
3,Sep 15 2022 10:05AM,246302,21,"NBTY Global, Inc.",1.0,0.0
4,Sep 15 2022 10:03AM,246301,21,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3204181,92.0,10.0
12,246493,10.0,4.0
16,739409,3.0,0.0
19,985949,4.0,2.0
20,246497,5.0,0.0
21,1969906,7.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3204181,92.0,10.0
1,12,246493,10.0,4.0
2,16,739409,3.0,0.0
3,19,985949,4.0,2.0
4,20,246497,5.0,0.0
5,21,1969906,7.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,92.0,10.0
1,12,10.0,4.0
2,16,3.0,0.0
3,19,4.0,2.0
4,20,5.0,0.0
5,21,7.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,92.0
1,12,Released,10.0
2,16,Released,3.0
3,19,Released,4.0
4,20,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Executing,10.0,4.0,0.0,2.0,0.0,1.0
Released,92.0,10.0,3.0,4.0,5.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Executing,10.0,4.0,0.0,2.0,0.0,1.0
1,Released,92.0,10.0,3.0,4.0,5.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Executing,10.0,4.0,0.0,2.0,0.0,1.0
1,Released,92.0,10.0,3.0,4.0,5.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()